In [1]:
from google.colab import drive
import os
import json
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore')

In [2]:
print("Mounting Google Drive...")
drive.mount('/content/drive')

Mounting Google Drive...
Mounted at /content/drive


In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ditogogua","key":"65a423eed3ea99d51a9c7348b802e85b"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions list | head -5

ref                                                                              deadline             category                reward  teamCount  userHasEntered  
-------------------------------------------------------------------------------  -------------------  ---------------  -------------  ---------  --------------  
https://www.kaggle.com/competitions/arc-prize-2025                               2025-11-03 23:59:00  Featured         1,000,000 Usd        640           False  
https://www.kaggle.com/competitions/google-gemma-3n-hackathon                    2025-08-06 23:59:00  Featured           150,000 Usd          0           False  
https://www.kaggle.com/competitions/make-data-count-finding-data-references      2025-09-09 23:59:00  Research           100,000 Usd        560           False  


In [6]:
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 796MB/s]


In [7]:
with zipfile.ZipFile('walmart-recruiting-store-sales-forecasting.zip', 'r') as zip_ref:
    zip_ref.extractall('walmart_data')

print("Extracted files:")
for root, dirs, files in os.walk('walmart_data'):
    for file in files:
        print(os.path.join(root, file))

Extracted files:
walmart_data/stores.csv
walmart_data/sampleSubmission.csv.zip
walmart_data/train.csv.zip
walmart_data/features.csv.zip
walmart_data/test.csv.zip


In [8]:
print("\nExtracting individual CSV files...")
csv_files = ['train.csv.zip', 'test.csv.zip', 'features.csv.zip', 'stores.csv', 'sampleSubmission.csv.zip']


Extracting individual CSV files...


In [9]:
for file in csv_files:
    file_path = os.path.join('walmart_data', file)
    if os.path.exists(file_path) and file.endswith('.zip'):
        print(f"Extracting {file}...")
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall('walmart_data')
    elif os.path.exists(file_path):
        print(f"{file} already extracted or not zipped")

Extracting train.csv.zip...
Extracting test.csv.zip...
Extracting features.csv.zip...
stores.csv already extracted or not zipped
Extracting sampleSubmission.csv.zip...


In [10]:
print("\nFinal extracted files:")
for root, dirs, files in os.walk('walmart_data'):
    for file in files:
        if file.endswith('.csv'):
            print(os.path.join(root, file))


Final extracted files:
walmart_data/train.csv
walmart_data/sampleSubmission.csv
walmart_data/stores.csv
walmart_data/test.csv
walmart_data/features.csv


In [11]:
!pip install statsmodels

from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error
from itertools import product

In [12]:
plt.style.use('default')
sns.set_palette("husl")

In [19]:
def compute_wmae(y_true, y_pred, is_holiday, holiday_weight=5):
    weights = np.where(is_holiday, holiday_weight, 1)
    denom = np.sum(weights)
    if denom == 0:
        return np.nan
    return np.sum(weights * np.abs(y_true - y_pred)) / denom


In [20]:
total_y_true, total_y_pred, total_is_holiday = [], [], []

val_weeks = int(len(walmart_arima.train_data) * 0.1)
val_data = walmart_arima.train_data.tail(val_weeks)

grouped = val_data.groupby(['Store', 'Dept'])

for (store, dept), group in grouped:
    group = group.sort_values('Date')
    ts = group.set_index('Date')['Weekly_Sales'].resample('W').sum().fillna(0)

    if len(ts) < 20:
        continue

    model = walmart_arima.simple_arima_fit(ts)
    if model is None:
        continue

    steps = len(ts)
    try:
        forecast, _ = walmart_arima.forecast_sales(model, steps=steps, title=f"Validation ({store}-{dept})")
    except:
        continue

    if len(forecast) != len(ts):
        continue

    total_y_true.extend(ts.values)
    total_y_pred.extend(forecast.values)

    holiday_series = group.groupby('Date')['IsHoliday'].max().resample('W').max().reindex(ts.index).fillna(False).astype(bool)
    total_is_holiday.extend(holiday_series.values)

y_true = np.array(total_y_true)
y_pred = np.array(total_y_pred)
is_holiday = np.array(total_is_holiday)

valid = ~np.isnan(y_true) & ~np.isnan(y_pred)
if valid.sum() == 0:
    print("❌ No valid data to compute WMAE")
else:
    wmae = compute_wmae(y_true[valid], y_pred[valid], is_holiday[valid])
    print(f"✅ Local Validation WMAE: {wmae:.2f}")



Generating forecasts for Validation (40-85)...

Generating forecasts for Validation (40-87)...

Generating forecasts for Validation (40-90)...

Generating forecasts for Validation (40-91)...

Generating forecasts for Validation (40-92)...

Generating forecasts for Validation (40-93)...

Generating forecasts for Validation (40-94)...

Generating forecasts for Validation (40-95)...

Generating forecasts for Validation (40-96)...

Generating forecasts for Validation (40-97)...

Generating forecasts for Validation (40-98)...

Generating forecasts for Validation (40-99)...

Generating forecasts for Validation (41-1)...

Generating forecasts for Validation (41-2)...

Generating forecasts for Validation (41-3)...

Generating forecasts for Validation (41-4)...

Generating forecasts for Validation (41-5)...


/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "



Generating forecasts for Validation (41-6)...

Generating forecasts for Validation (41-7)...

Generating forecasts for Validation (41-8)...

Generating forecasts for Validation (41-9)...

Generating forecasts for Validation (41-10)...

Generating forecasts for Validation (41-11)...

Generating forecasts for Validation (41-12)...


/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "



Generating forecasts for Validation (41-13)...

Generating forecasts for Validation (41-14)...

Generating forecasts for Validation (41-16)...

Generating forecasts for Validation (41-17)...

Generating forecasts for Validation (41-18)...

Generating forecasts for Validation (41-19)...

Generating forecasts for Validation (41-20)...

Generating forecasts for Validation (41-21)...


...


Generating forecasts for Validation (45-32)...

Generating forecasts for Validation (45-33)...

Generating forecasts for Validation (45-34)...

Generating forecasts for Validation (45-35)...

Generating forecasts for Validation (45-36)...

Generating forecasts for Validation (45-38)...

Generating forecasts for Validation (45-40)...

Generating forecasts for Validation (45-41)...

Generating forecasts for Validation (45-42)...

Generating forecasts for Validation (45-44)...

Generating forecasts for Validation (45-45)...

Generating forecasts for Validation (45-46)...


/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "



Generating forecasts for Validation (45-47)...

Generating forecasts for Validation (45-49)...

Generating forecasts for Validation (45-51)...

Generating forecasts for Validation (45-52)...

Generating forecasts for Validation (45-54)...

Generating forecasts for Validation (45-55)...

Generating forecasts for Validation (45-56)...

Generating forecasts for Validation (45-58)...

Generating forecasts for Validation (45-59)...

Generating forecasts for Validation (45-60)...

Generating forecasts for Validation (45-67)...

Generating forecasts for Validation (45-71)...

Generating forecasts for Validation (45-72)...

Generating forecasts for Validation (45-74)...

Generating forecasts for Validation (45-77)...

Generating forecasts for Validation (45-78)...

Generating forecasts for Validation (45-79)...

Generating forecasts for Validation (45-80)...

Generating forecasts for Validation (45-81)...

Generating forecasts for Validation (45-82)...

Generating forecasts for Validation (45

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "



Generating forecasts for Validation (45-96)...

Generating forecasts for Validation (45-97)...

Generating forecasts for Validation (45-98)...
✅ Local Validation WMAE: 2127.29


In [31]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')

def simple_arima_forecasting():
    """
    Simple, robust ARIMA-based Walmart forecasting approach
    """
    print("=" * 60)
    print("SIMPLE ARIMA FORECASTING - ROBUST APPROACH")
    print("=" * 60)

    print("Loading data...")
    train = pd.read_csv('walmart_data/train.csv')
    test = pd.read_csv('walmart_data/test.csv')
    stores = pd.read_csv('walmart_data/stores.csv')
    features = pd.read_csv('walmart_data/features.csv')

    train['Date'] = pd.to_datetime(train['Date'])
    test['Date'] = pd.to_datetime(test['Date'])
    features['Date'] = pd.to_datetime(features['Date'])

    train_merged = train.merge(stores, on='Store', how='left')
    train_merged = train_merged.merge(features, on=['Store', 'Date'], how='left')
    test_merged = test.merge(stores, on='Store', how='left')
    test_merged = test_merged.merge(features, on=['Store', 'Date'], how='left')

    if 'IsHoliday_x' in train_merged.columns and 'IsHoliday_y' in train_merged.columns:
        train_merged['IsHoliday'] = train_merged['IsHoliday_x'] | train_merged['IsHoliday_y']
        test_merged['IsHoliday'] = test_merged['IsHoliday_x'] | test_merged['IsHoliday_y']
        train_merged = train_merged.drop(['IsHoliday_x', 'IsHoliday_y'], axis=1)
        test_merged = test_merged.drop(['IsHoliday_x', 'IsHoliday_y'], axis=1)

    train_merged = train_merged.fillna(0)
    test_merged = test_merged.fillna(0)

    print("Creating validation split...")
    train_merged = train_merged.sort_values('Date')
    split_date = train_merged['Date'].max() - pd.DateOffset(weeks=13)

    train_val = train_merged[train_merged['Date'] <= split_date]
    val_data = train_merged[train_merged['Date'] > split_date]

    print(f"Training period: {train_val['Date'].min()} to {train_val['Date'].max()}")
    print(f"Validation period: {val_data['Date'].min()} to {val_data['Date'].max()}")

    holiday_multiplier = train_val[train_val['IsHoliday']]['Weekly_Sales'].mean() / train_val[~train_val['IsHoliday']]['Weekly_Sales'].mean()
    print(f"Holiday multiplier: {holiday_multiplier:.3f}")

    print("Creating simple forecasts with ARIMA adjustments...")

    val_predictions = []
    val_actuals = []
    val_holidays = []

    val_combinations = val_data.groupby(['Store', 'Dept']).size().reset_index()

    for _, row in val_combinations.iterrows():
        store = row['Store']
        dept = row['Dept']

        train_store_dept = train_val[(train_val['Store'] == store) & (train_val['Dept'] == dept)]
        val_store_dept = val_data[(val_data['Store'] == store) & (val_data['Dept'] == dept)]

        if len(train_store_dept) >= 10:
            train_store_dept = train_store_dept.sort_values('Date')
            train_store_dept.set_index('Date', inplace=True)

            ts = train_store_dept['Weekly_Sales'].resample('W').sum()
            ts = ts.fillna(method='ffill').fillna(0)

            if len(ts) >= 10:
                try:
                    model = ARIMA(ts, order=(1, 1, 1))
                    fitted_model = model.fit()
                    n_forecast = len(val_store_dept)
                    if n_forecast > 0:
                        forecast = fitted_model.forecast(steps=n_forecast)

                        for i, (idx, val_row) in enumerate(val_store_dept.iterrows()):
                            if i < len(forecast):
                                base_forecast = forecast[i]
                            else:
                                base_forecast = ts.mean()

                            if val_row['IsHoliday']:
                                forecast_value = base_forecast * holiday_multiplier
                            else:
                                forecast_value = base_forecast

                            forecast_value = max(0, forecast_value)
                            val_predictions.append(forecast_value)
                            val_actuals.append(val_row['Weekly_Sales'])
                            val_holidays.append(val_row['IsHoliday'])
                        continue

                except Exception as e:
                    print(f"ARIMA failed for store {store}, dept {dept}: {e}")

        if len(train_store_dept) > 0:
            base_forecast = train_store_dept['Weekly_Sales'].mean()
        else:
            base_forecast = train_val['Weekly_Sales'].mean()

        for idx, val_row in val_store_dept.iterrows():
            if val_row['IsHoliday']:
                forecast_value = base_forecast * holiday_multiplier
            else:
                forecast_value = base_forecast

            forecast_value = max(0, forecast_value)
            val_predictions.append(forecast_value)
            val_actuals.append(val_row['Weekly_Sales'])
            val_holidays.append(val_row['IsHoliday'])

    def compute_wmae(y_true, y_pred, is_holiday):
        weights = np.where(is_holiday, 5, 1)
        wmae = np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)
        return wmae

    wmae = compute_wmae(np.array(val_actuals), np.array(val_predictions), np.array(val_holidays))
    print(f"✅ Simple ARIMA Local Validation WMAE: {wmae:.2f}")

    print(f"\nValidation Statistics:")
    print(f"  Total validation rows: {len(val_actuals)}")
    print(f"  Holiday rows: {sum(val_holidays)}")
    print(f"  Non-holiday rows: {len(val_holidays) - sum(val_holidays)}")
    print(f"  Average actual sales: {np.mean(val_actuals):.2f}")
    print(f"  Average predicted sales: {np.mean(val_predictions):.2f}")

    return wmae

if __name__ == "__main__":
    wmae = simple_arima_forecasting()
    print(f"\n🎉 Final WMAE Score: {wmae:.2f}")

SIMPLE ARIMA FORECASTING - ROBUST APPROACH
Loading data...
Creating validation split...
Training period: 2010-02-05 00:00:00 to 2012-07-27 00:00:00
Validation period: 2012-08-03 00:00:00 to 2012-10-26 00:00:00
Holiday multiplier: 1.074
Creating simple forecasts with ARIMA adjustments...
✅ Simple ARIMA Local Validation WMAE: 2213.76

Validation Statistics:
  Total validation rows: 38530
  Holiday rows: 2966
  Non-holiday rows: 35564
  Average actual sales: 15620.50
  Average predicted sales: 15948.62

🎉 Final WMAE Score: 2213.76


In [32]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')

def optimized_arima_forecasting():
    """
    Optimized ARIMA-based Walmart forecasting approach to get WMAE below 2000
    """
    print("=" * 60)
    print("OPTIMIZED ARIMA FORECASTING - TARGET WMAE < 2000")
    print("=" * 60)

    print("Loading data...")
    train = pd.read_csv('walmart_data/train.csv')
    test = pd.read_csv('walmart_data/test.csv')
    stores = pd.read_csv('walmart_data/stores.csv')
    features = pd.read_csv('walmart_data/features.csv')

    train['Date'] = pd.to_datetime(train['Date'])
    test['Date'] = pd.to_datetime(test['Date'])
    features['Date'] = pd.to_datetime(features['Date'])

    train_merged = train.merge(stores, on='Store', how='left')
    train_merged = train_merged.merge(features, on=['Store', 'Date'], how='left')
    test_merged = test.merge(stores, on='Store', how='left')
    test_merged = test_merged.merge(features, on=['Store', 'Date'], how='left')

    if 'IsHoliday_x' in train_merged.columns and 'IsHoliday_y' in train_merged.columns:
        train_merged['IsHoliday'] = train_merged['IsHoliday_x'] | train_merged['IsHoliday_y']
        test_merged['IsHoliday'] = test_merged['IsHoliday_x'] | test_merged['IsHoliday_y']
        train_merged = train_merged.drop(['IsHoliday_x', 'IsHoliday_y'], axis=1)
        test_merged = test_merged.drop(['IsHoliday_x', 'IsHoliday_y'], axis=1)

    train_merged = train_merged.fillna(0)
    test_merged = test_merged.fillna(0)

    print("Creating enhanced features...")

    train_merged['Month'] = train_merged['Date'].dt.month
    train_merged['Week'] = train_merged['Date'].dt.isocalendar().week
    train_merged['DayOfYear'] = train_merged['Date'].dt.dayofyear

    store_dept_stats = train_merged.groupby(['Store', 'Dept']).agg({
        'Weekly_Sales': ['mean', 'std', 'median', 'min', 'max'],
        'Temperature': 'mean',
        'Fuel_Price': 'mean',
        'CPI': 'mean',
        'Unemployment': 'mean'
    }).reset_index()

    store_dept_stats.columns = ['Store', 'Dept', 'Sales_Mean', 'Sales_Std', 'Sales_Median',
                               'Sales_Min', 'Sales_Max', 'Temp_Mean', 'Fuel_Mean', 'CPI_Mean', 'Unemp_Mean']

    train_merged = train_merged.merge(store_dept_stats, on=['Store', 'Dept'], how='left')
    test_merged = test_merged.merge(store_dept_stats, on=['Store', 'Dept'], how='left')

    print("Creating validation split...")
    train_merged = train_merged.sort_values('Date')
    split_date = train_merged['Date'].max() - pd.DateOffset(weeks=13)

    train_val = train_merged[train_merged['Date'] <= split_date]
    val_data = train_merged[train_merged['Date'] > split_date]

    print(f"Training period: {train_val['Date'].min()} to {train_val['Date'].max()}")
    print(f"Validation period: {val_data['Date'].min()} to {val_data['Date'].max()}")

    print("Calculating store-dept specific holiday multipliers...")
    holiday_multipliers = {}

    for store in train_val['Store'].unique():
        for dept in train_val[train_val['Store'] == store]['Dept'].unique():
            store_dept_data = train_val[(train_val['Store'] == store) & (train_val['Dept'] == dept)]
            if len(store_dept_data) > 0:
                holiday_data = store_dept_data[store_dept_data['IsHoliday']]
                non_holiday_data = store_dept_data[~store_dept_data['IsHoliday']]

                if len(holiday_data) > 0 and len(non_holiday_data) > 0:
                    holiday_avg = holiday_data['Weekly_Sales'].mean()
                    non_holiday_avg = non_holiday_data['Weekly_Sales'].mean()
                    if non_holiday_avg > 0:
                        holiday_multipliers[(store, dept)] = holiday_avg / non_holiday_avg
                    else:
                        holiday_multipliers[(store, dept)] = 1.2
                else:
                    holiday_multipliers[(store, dept)] = 1.2
            else:
                holiday_multipliers[(store, dept)] = 1.2

    global_holiday_multiplier = train_val[train_val['IsHoliday']]['Weekly_Sales'].mean() / train_val[~train_val['IsHoliday']]['Weekly_Sales'].mean()
    print(f"Global holiday multiplier: {global_holiday_multiplier:.3f}")

    print("Creating optimized forecasts...")

    val_predictions = []
    val_actuals = []
    val_holidays = []

    val_combinations = val_data.groupby(['Store', 'Dept']).size().reset_index()

    for _, row in val_combinations.iterrows():
        store = row['Store']
        dept = row['Dept']

        train_store_dept = train_val[(train_val['Store'] == store) & (train_val['Dept'] == dept)]
        val_store_dept = val_data[(val_data['Store'] == store) & (val_data['Dept'] == dept)]

        if len(train_store_dept) >= 15:
            train_store_dept = train_store_dept.sort_values('Date')
            train_store_dept.set_index('Date', inplace=True)

            ts = train_store_dept['Weekly_Sales'].resample('W').sum()
            ts = ts.fillna(method='ffill').fillna(0)

            if len(ts) >= 15:
                try:
                    configs = [(1, 1, 1), (2, 1, 1), (1, 1, 2), (0, 1, 1), (1, 0, 1), (2, 1, 2)]
                    best_aic = float('inf')
                    best_model = None

                    for order in configs:
                        try:
                            model = ARIMA(ts, order=order)
                            fitted = model.fit()
                            if fitted.aic < best_aic:
                                best_aic = fitted.aic
                                best_model = fitted
                        except:
                            continue

                    if best_model is not None:
                        n_forecast = len(val_store_dept)
                        if n_forecast > 0:
                            forecast = best_model.forecast(steps=n_forecast)

                            for i, (idx, val_row) in enumerate(val_store_dept.iterrows()):
                                if i < len(forecast):
                                    base_forecast = forecast[i]
                                else:
                                    base_forecast = ts.mean()

                                holiday_mult = holiday_multipliers.get((store, dept), global_holiday_multiplier)

                                if val_row['IsHoliday']:
                                    forecast_value = base_forecast * holiday_mult
                                else:
                                    forecast_value = base_forecast

                                if val_row['Temperature'] > 80:
                                    forecast_value *= 0.95
                                elif val_row['Temperature'] < 30:
                                    forecast_value *= 1.05

                                forecast_value = max(0, forecast_value)
                                val_predictions.append(forecast_value)
                                val_actuals.append(val_row['Weekly_Sales'])
                                val_holidays.append(val_row['IsHoliday'])
                            continue

                except Exception as e:
                    print(f"ARIMA failed for store {store}, dept {dept}: {e}")

        if len(train_store_dept) > 0:
            train_store_dept['Month'] = train_store_dept['Date'].dt.month
            monthly_avg = train_store_dept.groupby('Month')['Weekly_Sales'].mean()

            recent_data = train_store_dept.tail(20)
            recent_avg = recent_data['Weekly_Sales'].mean()

            base_forecast = 0.7 * recent_avg + 0.3 * train_store_dept['Weekly_Sales'].mean()
        else:
            base_forecast = train_val['Weekly_Sales'].mean()

        for idx, val_row in val_store_dept.iterrows():
            holiday_mult = holiday_multipliers.get((store, dept), global_holiday_multiplier)

            if val_row['IsHoliday']:
                forecast_value = base_forecast * holiday_mult
            else:
                forecast_value = base_forecast

            if val_row['Temperature'] > 80:
                forecast_value *= 0.95
            elif val_row['Temperature'] < 30:
                forecast_value *= 1.05

            forecast_value = max(0, forecast_value)
            val_predictions.append(forecast_value)
            val_actuals.append(val_row['Weekly_Sales'])
            val_holidays.append(val_row['IsHoliday'])

    def compute_wmae(y_true, y_pred, is_holiday):
        weights = np.where(is_holiday, 5, 1)
        wmae = np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)
        return wmae

    wmae = compute_wmae(np.array(val_actuals), np.array(val_predictions), np.array(val_holidays))
    print(f"✅ Optimized ARIMA Local Validation WMAE: {wmae:.2f}")

    print(f"\nValidation Statistics:")
    print(f"  Total validation rows: {len(val_actuals)}")
    print(f"  Holiday rows: {sum(val_holidays)}")
    print(f"  Non-holiday rows: {len(val_holidays) - sum(val_holidays)}")
    print(f"  Average actual sales: {np.mean(val_actuals):.2f}")
    print(f"  Average predicted sales: {np.mean(val_predictions):.2f}")
    print(f"  MAE (non-weighted): {np.mean(np.abs(np.array(val_actuals) - np.array(val_predictions))):.2f}")

    return wmae

if __name__ == "__main__":
    wmae = optimized_arima_forecasting()
    print(f"\n🎉 Final WMAE Score: {wmae:.2f}")
    if wmae < 2000:
        print("🎯 SUCCESS! WMAE is below 2000!")
    else:
        print("📈 Close! Try running again or further optimizations needed.")

OPTIMIZED ARIMA FORECASTING - TARGET WMAE < 2000
Loading data...
Creating enhanced features...
Creating validation split...
Training period: 2010-02-05 00:00:00 to 2012-07-27 00:00:00
Validation period: 2012-08-03 00:00:00 to 2012-10-26 00:00:00
Calculating store-dept specific holiday multipliers...
Global holiday multiplier: 1.074
Creating optimized forecasts...
✅ Optimized ARIMA Local Validation WMAE: 2550.43

Validation Statistics:
  Total validation rows: 38530
  Holiday rows: 2966
  Non-holiday rows: 35564
  Average actual sales: 15620.50
  Average predicted sales: 15819.96
  MAE (non-weighted): 2186.85

🎉 Final WMAE Score: 2550.43
📈 Close! Try running again or further optimizations needed.


In [33]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')

def fast_optimized_arima_forecasting():
    """
    Fast optimized ARIMA-based Walmart forecasting approach
    """
    print("=" * 60)
    print("FAST OPTIMIZED ARIMA FORECASTING")
    print("=" * 60)

    print("Loading data...")
    train = pd.read_csv('walmart_data/train.csv')
    test = pd.read_csv('walmart_data/test.csv')
    stores = pd.read_csv('walmart_data/stores.csv')
    features = pd.read_csv('walmart_data/features.csv')

    train['Date'] = pd.to_datetime(train['Date'])
    test['Date'] = pd.to_datetime(test['Date'])
    features['Date'] = pd.to_datetime(features['Date'])

    train_merged = train.merge(stores, on='Store', how='left')
    train_merged = train_merged.merge(features, on=['Store', 'Date'], how='left')
    test_merged = test.merge(stores, on='Store', how='left')
    test_merged = test_merged.merge(features, on=['Store', 'Date'], how='left')

    if 'IsHoliday_x' in train_merged.columns and 'IsHoliday_y' in train_merged.columns:
        train_merged['IsHoliday'] = train_merged['IsHoliday_x'] | train_merged['IsHoliday_y']
        test_merged['IsHoliday'] = test_merged['IsHoliday_x'] | test_merged['IsHoliday_y']
        train_merged = train_merged.drop(['IsHoliday_x', 'IsHoliday_y'], axis=1)
        test_merged = test_merged.drop(['IsHoliday_x', 'IsHoliday_y'], axis=1)

    train_merged = train_merged.fillna(0)
    test_merged = test_merged.fillna(0)

    print("Creating validation split...")
    train_merged = train_merged.sort_values('Date')
    split_date = train_merged['Date'].max() - pd.DateOffset(weeks=13)

    train_val = train_merged[train_merged['Date'] <= split_date]
    val_data = train_merged[train_merged['Date'] > split_date]

    print(f"Training period: {train_val['Date'].min()} to {train_val['Date'].max()}")
    print(f"Validation period: {val_data['Date'].min()} to {val_data['Date'].max()}")

    holiday_multiplier = train_val[train_val['IsHoliday']]['Weekly_Sales'].mean() / train_val[~train_val['IsHoliday']]['Weekly_Sales'].mean()
    print(f"Holiday multiplier: {holiday_multiplier:.3f}")

    print("Creating fast optimized forecasts...")

    val_predictions = []
    val_actuals = []
    val_holidays = []

    val_combinations = val_data.groupby(['Store', 'Dept']).size().reset_index()
    print(f"Processing {len(val_combinations)} store-dept combinations...")

    for idx, row in val_combinations.iterrows():
        if idx % 10 == 0:
            print(f"Processing combination {idx+1}/{len(val_combinations)}")

        store = row['Store']
        dept = row['Dept']

        train_store_dept = train_val[(train_val['Store'] == store) & (train_val['Dept'] == dept)]
        val_store_dept = val_data[(val_data['Store'] == store) & (val_data['Dept'] == dept)]

        if len(train_store_dept) >= 12:
            train_store_dept = train_store_dept.sort_values('Date')
            train_store_dept.set_index('Date', inplace=True)

            ts = train_store_dept['Weekly_Sales'].resample('W').sum()
            ts = ts.fillna(method='ffill').fillna(0)

            if len(ts) >= 12:
                try:
                    configs = [(1, 1, 1), (2, 1, 1), (1, 1, 2)]
                    best_aic = float('inf')
                    best_model = None

                    for order in configs:
                        try:
                            model = ARIMA(ts, order=order)
                            fitted = model.fit()
                            if fitted.aic < best_aic:
                                best_aic = fitted.aic
                                best_model = fitted
                        except:
                            continue

                    if best_model is not None:
                        n_forecast = len(val_store_dept)
                        if n_forecast > 0:
                            forecast = best_model.forecast(steps=n_forecast)

                            for i, (idx, val_row) in enumerate(val_store_dept.iterrows()):
                                if i < len(forecast):
                                    base_forecast = forecast[i]
                                else:
                                    base_forecast = ts.mean()

                                if val_row['IsHoliday']:
                                    forecast_value = base_forecast * holiday_multiplier
                                else:
                                    forecast_value = base_forecast

                                if val_row['Temperature'] > 80:
                                    forecast_value *= 0.98
                                elif val_row['Temperature'] < 30:
                                    forecast_value *= 1.02

                                forecast_value = max(0, forecast_value)
                                val_predictions.append(forecast_value)
                                val_actuals.append(val_row['Weekly_Sales'])
                                val_holidays.append(val_row['IsHoliday'])
                            continue

                except Exception as e:
                    pass

        if len(train_store_dept) > 0:
            recent_data = train_store_dept.tail(15)
            recent_avg = recent_data['Weekly_Sales'].mean()
            overall_avg = train_store_dept['Weekly_Sales'].mean()

            base_forecast = 0.8 * recent_avg + 0.2 * overall_avg
        else:
            base_forecast = train_val['Weekly_Sales'].mean()

        for idx, val_row in val_store_dept.iterrows():
            if val_row['IsHoliday']:
                forecast_value = base_forecast * holiday_multiplier
            else:
                forecast_value = base_forecast

            if val_row['Temperature'] > 80:
                forecast_value *= 0.98
            elif val_row['Temperature'] < 30:
                forecast_value *= 1.02

            forecast_value = max(0, forecast_value)
            val_predictions.append(forecast_value)
            val_actuals.append(val_row['Weekly_Sales'])
            val_holidays.append(val_row['IsHoliday'])

    def compute_wmae(y_true, y_pred, is_holiday):
        weights = np.where(is_holiday, 5, 1)
        wmae = np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)
        return wmae

    wmae = compute_wmae(np.array(val_actuals), np.array(val_predictions), np.array(val_holidays))
    print(f"✅ Fast Optimized ARIMA Local Validation WMAE: {wmae:.2f}")

    print(f"\nValidation Statistics:")
    print(f"  Total validation rows: {len(val_actuals)}")
    print(f"  Holiday rows: {sum(val_holidays)}")
    print(f"  Non-holiday rows: {len(val_holidays) - sum(val_holidays)}")
    print(f"  Average actual sales: {np.mean(val_actuals):.2f}")
    print(f"  Average predicted sales: {np.mean(val_predictions):.2f}")

    return wmae

if __name__ == "__main__":
    wmae = fast_optimized_arima_forecasting()
    print(f"\n🎉 Final WMAE Score: {wmae:.2f}")
    if wmae < 2000:
        print("🎯 SUCCESS! WMAE is below 2000!")
    else:
        print("📈 Close! Try running again or further optimizations needed.")

FAST OPTIMIZED ARIMA FORECASTING
Loading data...
Creating validation split...
Training period: 2010-02-05 00:00:00 to 2012-07-27 00:00:00
Validation period: 2012-08-03 00:00:00 to 2012-10-26 00:00:00
Holiday multiplier: 1.074
Creating fast optimized forecasts...
Processing 3101 store-dept combinations...
Processing combination 1/3101
Processing combination 11/3101
Processing combination 21/3101
Processing combination 31/3101
Processing combination 41/3101
Processing combination 51/3101
Processing combination 61/3101
Processing combination 71/3101
Processing combination 81/3101
Processing combination 91/3101
Processing combination 101/3101
Processing combination 111/3101
Processing combination 121/3101
Processing combination 131/3101
Processing combination 141/3101
Processing combination 151/3101
Processing combination 161/3101
Processing combination 171/3101
Processing combination 181/3101
Processing combination 191/3101
Processing combination 201/3101
Processing combination 211/3101
P